In [17]:
import pandas as pd

In [95]:
df_sales = pd.read_csv("sales_train_validation.csv")
df_prices = pd.read_csv("sell_prices.csv")
df_calendar = pd.read_csv("calendar.csv")

In [239]:
def generate_store_df(store_cod):
    df_sales_AUX = df_sales[df_sales["store_id"] == store_cod].copy()
    df_prices_AUX = df_prices[df_prices["store_id"] == store_cod].copy()
    df_calendar_AUX = df_calendar.copy()
    
    df_sales_AUX.drop(["id", "dept_id", "cat_id", "store_id", "state_id"], axis=1, inplace=True)
    df_calendar_AUX = df_calendar_AUX[["wm_yr_wk","date", "d", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]]
    
    df_prices_AUX = df_prices_AUX.pivot(index="wm_yr_wk", columns="item_id", values="sell_price")
    df_prices_AUX.fillna(method="bfill", axis=1, inplace=True)
    
    df_calendar_AUX = df_calendar_AUX.merge(df_prices_CA_1_pivoted, on="wm_yr_wk", how="left")
    df_calendar_AUX.drop("wm_yr_wk", axis=1, inplace=True)
#     df_calendar_AUX["d"] = df_calendar_AUX["d"].apply(lambda x : int(x.split("_")[1]))
    
    sales_days = ["d_"+str(x) for x in range(1, 1914)]
    df_sales_AUX = pd.melt(df_sales_AUX, id_vars=["item_id"], value_vars=sales_days)
    df_sales_AUX.rename(columns = {'variable':'d'}, inplace=True)
    df_sales_AUX = df_sales_AUX.pivot(index='d', columns='item_id', values='value')
    df_sales_AUX.reset_index(inplace=True)
#     df_sales_CA_1_pivoted["d"] = df_sales_AUX["d"].apply(lambda x : int(x.split("_")[1]))
    
    df_d_to_date = df_calendar[["date", "d"]].copy()
#     df_d_to_date["d"] = df_d_to_date["d"].apply(lambda x : int(x.split("_")[1]))
    
    
    result = df_sales_AUX.merge(df_calendar_AUX, on="d", how="left")
    result = result.merge(df_d_to_date, on="d", how="left")
    result.rename(columns = {'date_y':'date'}, inplace=True)
    result["d"] = result["d"].apply(lambda x : int(x.split("_")[1]))
    result.set_index("d", drop=True, inplace=True)
    result.sort_index(inplace=True)
    result.columns = result.columns.str.replace("_x", "_sells")
    result.columns = result.columns.str.replace("_y", "_price")
    
#     print(result.head())
    return result

In [240]:
r = generate_store_df("CA_1")

In [241]:
s = generate_store_df("CA_2")

In [242]:
s.head()

,FOODS_1_001_sells,FOODS_1_002_sells,FOODS_1_003_sells,FOODS_1_004_sells,FOODS_1_005_sells,FOODS_1_006_sells,FOODS_1_008_sells,FOODS_1_009_sells,FOODS_1_010_sells,FOODS_1_011_sells,...,HOUSEHOLD_2_508_price,HOUSEHOLD_2_509_price,HOUSEHOLD_2_510_price,HOUSEHOLD_2_511_price,HOUSEHOLD_2_512_price,HOUSEHOLD_2_513_price,HOUSEHOLD_2_514_price,HOUSEHOLD_2_515_price,HOUSEHOLD_2_516_price,date
d,,,,,,,,,,,,,,,,,,,,,
1,2,2,8,0,8,1,0,0,0,0,...,8.97,8.97,2.87,2.87,3.97,18.97,18.97,5.94,5.94,2011-01-29
2,0,1,2,0,1,1,0,0,0,0,...,8.97,8.97,2.87,2.87,3.97,18.97,18.97,5.94,5.94,2011-01-30
3,0,1,2,0,1,1,0,0,0,0,...,8.97,8.97,2.87,2.87,3.97,18.97,18.97,5.94,5.94,2011-01-31
4,0,0,1,0,0,3,0,0,0,0,...,8.97,8.97,2.87,2.87,3.97,18.97,18.97,5.94,5.94,2011-02-01
5,1,2,0,0,5,0,0,0,0,0,...,8.97,8.97,2.87,2.87,3.97,18.97,18.97,5.94,5.94,2011-02-02
